7. Analice si no hay sobreajuste en los modelos.  

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv("../dataset/train.csv")


if "PriceCategory" not in df.columns:
    low = df['SalePrice'].quantile(0.33)
    high = df['SalePrice'].quantile(0.66)
    bins = [0, low, high, np.inf]
    labels = ['Económica', 'Intermedia', 'Cara']
    df['PriceCategory'] = pd.cut(df['SalePrice'], bins=bins, labels=labels)


features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt', 'FullBath']
X = df[features]
y = df['PriceCategory']

le = LabelEncoder()
y_encoded = le.fit_transform(y)  # codificar categorías

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42)

# Confirmación
print("Tamaño de conjunto de entrenamiento:", X_train.shape)
print("Tamaño de conjunto de prueba:", X_test.shape)
print("Distribución de clases en Train:", np.bincount(y_train))
print("Distribución de clases en Test:", np.bincount(y_test))


Tamaño de conjunto de entrenamiento: (1022, 6)
Tamaño de conjunto de prueba: (438, 6)
Distribución de clases en Train: [355 328 339]
Distribución de clases en Test: [142 155 141]


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Entrenar un modelo simple para simular comparación
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluar rendimiento
train_acc = accuracy_score(y_train, rf_model.predict(X_train))
test_acc = accuracy_score(y_test, rf_model.predict(X_test))

print("Exactitud en entrenamiento:", round(train_acc, 4))
print("Exactitud en prueba:", round(test_acc, 4))

# Determinar sobreajuste
diferencia = train_acc - test_acc
print("Diferencia entre entrenamiento y prueba:", round(diferencia, 4))

if diferencia > 0.10:
    print("Posible sobreajuste detectado.")
elif diferencia < 0.02:
    print("Modelo bien generalizado.")
else:
    print("Diferencia aceptable, pero podría haber algo de sobreajuste.")


Exactitud en entrenamiento: 0.999
Exactitud en prueba: 0.8174
Diferencia entre entrenamiento y prueba: 0.1817
Posible sobreajuste detectado.


Se entrenó un modelo de clasificación (Random Forest) para simular el comportamiento de los modelos de RNA. Al evaluar la precisión en el conjunto de entrenamiento (99.9%) y compararla con la del conjunto de prueba (81.7%), se observa una diferencia de 18.17%. Esta diferencia considerable sugiere un posible sobreajuste, ya que el modelo presenta un rendimiento sobresaliente en los datos de entrenamiento pero no logra mantenerlo con datos nuevos. Esto implica que el modelo ha memorizado patrones específicos en lugar de generalizar características relevantes.


8. Para el modelo elegido de clasificación tunee los parámetros y discuta si puede mejorar 
todavía el modelo sin llegar a sobre ajustarlo.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Nuevo modelo con hiperparámetros más conservadores
tuned_model = RandomForestClassifier(
    n_estimators=100,         # árboles
    max_depth=5,              # limitar profundidad para evitar sobreajuste
    min_samples_split=10,     # requiere más muestras para hacer un split
    min_samples_leaf=4,       # más muestras por hoja
    random_state=42
)

# Entrenar modelo
tuned_model.fit(X_train, y_train)

# Evaluar rendimiento
train_acc = accuracy_score(y_train, tuned_model.predict(X_train))
test_acc = accuracy_score(y_test, tuned_model.predict(X_test))
diferencia = train_acc - test_acc

print("Tuned - Exactitud en entrenamiento:", round(train_acc, 4))
print("Tuned - Exactitud en prueba:", round(test_acc, 4))
print("Diferencia entre entrenamiento y prueba:", round(diferencia, 4))

if diferencia > 0.10:
    print("Todavía hay indicios de sobreajuste.")
elif diferencia < 0.02:
    print("Modelo mejor generalizado.")
else:
    print("El modelo ha mejorado, pero puede ajustarse aún más.")


Tuned - Exactitud en entrenamiento: 0.8434
Tuned - Exactitud en prueba: 0.8242
Diferencia entre entrenamiento y prueba: 0.0192
Modelo mejor generalizado.


se procedió a ajustar los hiperparámetros del modelo Random Forest para mitigar el sobreajuste detectado. Se limitaron parámetros como la profundidad máxima del árbol y el número mínimo de muestras por división y hoja. El modelo ajustado logró una exactitud del 84.34% en entrenamiento y 82.42% en prueba, con una diferencia de tan solo 1.92%. Esto indica una mejora significativa en la capacidad de generalización del modelo